In [7]:
import numpy as np
import cv2
import dlib
import imutils
from imutils import face_utils
import math

In [15]:
def distance(x1,y1,x2,y2):
    """Calculate the distance between two points"""
    d = math.sqrt(math.pow((x2-x1),2) + math.pow((y2-y1),2))
    return d

In [19]:
def EAR(land_dict):
    """Calculate the Eye aspect ratio"""
    dist = {}
    dist.update({'d1':distance(land_dict[36][0],land_dict[36][1],land_dict[39][0],land_dict[39][1])})
    dist.update({'d2':distance(land_dict[37][0],land_dict[37][1],land_dict[41][0],land_dict[41][1])})
    dist.update({'d3':distance(land_dict[38][0],land_dict[38][1],land_dict[40][0],land_dict[40][1])})
    dist.update({'d4':distance(land_dict[42][0],land_dict[42][1],land_dict[45][0],land_dict[45][1])})
    dist.update({'d5':distance(land_dict[43][0],land_dict[43][1],land_dict[47][0],land_dict[47][1])})
    dist.update({'d6':distance(land_dict[44][0],land_dict[44][1],land_dict[46][0],land_dict[46][1])})
    Ear1 = (dist['d2'] + dist['d3'])/(2*dist['d1'])
    Ear2 = (dist['d5'] + dist['d6'])/(2*dist['d4'])
    return Ear1,Ear2

In [31]:
cam = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector() #to detect faces on the frame
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") #to detect landmarks on the face(0-67)point representation
frame_count = 0
while(True):
    ret,img = cam.read()
    grayimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    faces = detector(grayimg) #list of coordinates of a rectangle where face is
    #for eg face = rectangles[[(342(x1), 231(y1)) (466(x2), 355(y2))]]
    for face in faces:
        landmarks = predictor(grayimg,face)
        land_dict = {}
        for i in range(36,48,1):
            x = landmarks.part(i).x
            y = landmarks.part(i).y
            land_dict.update({i:[x,y]})
            #print(x,y)
            #cv2.circle(img,(x,y),3,(255,0,0),-1)
        E1,E2 = EAR(land_dict)
        #print(E1,E2)
        if (E1 and E2) <0.25:
            frame_count+=1
            if frame_count>20:
                cv2.putText(img, "****************ALERT!****************", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(img, "****************ALERT!****************", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            frame_count = 0
    cv2.imshow("Image",img)
    if(cv2.waitKey(1)==ord('q')):
        break

cam.release()        
cv2.destroyAllWindows()

